In [1]:
import json
import numpy as np
from utils.transform_utils import quat2mat, convert_quat
import cv2
from scipy.spatial.transform import Rotation

import sys
sys.path.append("/home/vdrame/catkin_ws/src/py_panda/PyPanda")
from PyPanda import Robot
import rospy

from utils.camera_utils import RealCamera, RealCameraROS

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-06-26 17:17:58,044 - utils - NumExpr defaulting to 8 threads.
INFO - 2023-06-26 17:17:58,425 - topics - topicmanager initialized


# viewpoint registering

In [3]:
rospy.init_node("pythonnode_register_views",anonymous=True)
rbt = Robot("panda", use_gripper=True)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

# views_pos = []
# views_orn_wxyz = []
views = {}
view_idx = 0
while True:
    bgr, depth_image, depth_scale = camera.get_rgb_depth()
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    cv2.imshow("rgb", rgb)
    key = cv2.waitKey(1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    elif key == ord('s'):
        view_pos = rbt.model.ee_pos_rel()
        view_orn_wxyz = rbt.model.ee_orn_rel()
        views["view"+str(view_idx)] = {"pos": view_pos.tolist(), "orn_wxyz": view_orn_wxyz.tolist()}
        print("view{} saved".format(view_idx))
        view_idx += 1


print(views)

#write pos in json file
with open('config/views_pos.json', 'w') as outfile:
    json.dump(views, outfile, indent=1)

[WARN] [1687792753.518808]: No controllers are running


rospy already initialized
Camera topic found
view0 saved
view1 saved
view2 saved
view3 saved
view4 saved
{'view0': {'pos': [0.23985722240303245, 0.26052195515410653, 0.5701068144273241], 'orn_wxyz': [-0.005912699137142532, 0.9536396433375781, 0.3008391095076146, 0.005683382316541577]}, 'view1': {'pos': [0.07949707721277763, 0.22859862050453356, 0.45097533839767734], 'orn_wxyz': [-0.036899163509989986, 0.9394326958337057, 0.32285019038500107, 0.10896061811153543]}, 'view2': {'pos': [0.3915209392408166, 0.4862067957111822, 0.32073808016367844], 'orn_wxyz': [0.08162255205929227, 0.905821251370701, 0.3135523983046636, -0.27296613906845485]}, 'view3': {'pos': [0.052530708367927836, 0.6078261628010504, 0.3588777706481084], 'orn_wxyz': [0.1711059617949872, 0.9289587831960118, -0.239970386065929, 0.2239922828415443]}, 'view4': {'pos': [0.5704656790529168, 0.20563119513338096, 0.29730371651941423], 'orn_wxyz': [-0.32403252617643036, 0.5311144962276173, 0.7779446310577263, 0.0878775562078473]}}


In [4]:
#load pos and orn from json file
with open('config/views_pos.json') as json_file:
    views_pos_1 = json.load(json_file)

keys = views_pos_1.keys()
for key in keys:
    print(key)
    print(views_pos_1[key]["pos"])
    print(views_pos_1[key]["orn_wxyz"])
    print("\n")

view0
[0.23985722240303245, 0.26052195515410653, 0.5701068144273241]
[-0.005912699137142532, 0.9536396433375781, 0.3008391095076146, 0.005683382316541577]


view1
[0.07949707721277763, 0.22859862050453356, 0.45097533839767734]
[-0.036899163509989986, 0.9394326958337057, 0.32285019038500107, 0.10896061811153543]


view2
[0.3915209392408166, 0.4862067957111822, 0.32073808016367844]
[0.08162255205929227, 0.905821251370701, 0.3135523983046636, -0.27296613906845485]


view3
[0.052530708367927836, 0.6078261628010504, 0.3588777706481084]
[0.1711059617949872, 0.9289587831960118, -0.239970386065929, 0.2239922828415443]


view4
[0.5704656790529168, 0.20563119513338096, 0.29730371651941423]
[-0.32403252617643036, 0.5311144962276173, 0.7779446310577263, 0.0878775562078473]




# grid_search

In [4]:
rospy.init_node("pythonnode_register_views",anonymous=True)
rbt = Robot("panda", use_gripper=True)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

# views_pos = []
# views_orn_wxyz = []
views = {}
view_idx = 0
while True:
    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    cv2.imshow("rgb", rgb)
    key = cv2.waitKey(1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    elif key == ord('s'):
        view_pos = rbt.model.ee_pos_rel()
        view_orn_wxyz = rbt.model.ee_orn_rel()
        views["view"+str(view_idx)] = {"pos": view_pos.tolist(), "orn_wxyz": view_orn_wxyz.tolist()}
        print("view{} saved".format(view_idx))
        view_idx += 1


print(views)

#write pos in json file
with open('config/gridSearch_pos.json', 'w') as outfile:
    json.dump(views, outfile, indent=1)

[WARN] [1687261933.082417]: No controllers are running


rospy already initialized
Camera topic found
view0 saved
view1 saved
view2 saved
view3 saved
view4 saved
{'view0': {'pos': [0.2014731861308635, 0.27728404957761793, 0.5691502058214037], 'orn_wxyz': [0.011270544156379758, 0.9453026937389689, 0.32584943030574887, -0.009896505124897762]}, 'view1': {'pos': [0.15176439988072532, 0.38683435889267503, 0.6216839474523348], 'orn_wxyz': [-0.11931739752775403, 0.9457810334614712, 0.29489491880963725, -0.06556357412475868]}, 'view2': {'pos': [0.0208210622695692, 0.5437262275203616, 0.588480050095034], 'orn_wxyz': [-0.003286465811994746, 0.9547804783455706, 0.2948399007736562, -0.03811653479069696]}, 'view3': {'pos': [0.3421814487635199, 0.16826035922133037, 0.5996855900321988], 'orn_wxyz': [0.24775681490706572, 0.8912595297570436, 0.3658964320580957, 0.10194514354060673]}, 'view4': {'pos': [0.29169241183965483, 0.33434424780146604, 0.6105792403316133], 'orn_wxyz': [-0.10046322474704415, 0.9220701341447687, 0.3293129674125185, 0.17676758098032194]}

In [5]:
#load pos and orn from json file
with open('config/gridSearch_pos.json') as json_file:
    grid_view = json.load(json_file)

keys = grid_view.keys()
for key in keys:
    print(key)
    print(grid_view[key]["pos"])
    print(grid_view[key]["orn_wxyz"])
    print("\n")

view0
[0.2014731861308635, 0.27728404957761793, 0.5691502058214037]
[0.011270544156379758, 0.9453026937389689, 0.32584943030574887, -0.009896505124897762]


view1
[0.15176439988072532, 0.38683435889267503, 0.6216839474523348]
[-0.11931739752775403, 0.9457810334614712, 0.29489491880963725, -0.06556357412475868]


view2
[0.0208210622695692, 0.5437262275203616, 0.588480050095034]
[-0.003286465811994746, 0.9547804783455706, 0.2948399007736562, -0.03811653479069696]


view3
[0.3421814487635199, 0.16826035922133037, 0.5996855900321988]
[0.24775681490706572, 0.8912595297570436, 0.3658964320580957, 0.10194514354060673]


view4
[0.29169241183965483, 0.33434424780146604, 0.6105792403316133]
[-0.10046322474704415, 0.9220701341447687, 0.3293129674125185, 0.17676758098032194]




# Write camera extrinsic

In [10]:
#D415
D415_hand2camera_pos = np.array([0.0488546636437146,-0.03384417860749521,0.0512776975002817]) 
D415_hand2camera_quat = [0.012961267509189803,-0.0012768531849757236,0.7052247395136084,0.708864191484139] #xyzw 
D415_intrinsic = np.array([[925,    0.,           626],
                           [0.,         926,      345],
                           [0.,         0.,           1.]])
#L515
L515_hand2camera_pos = np.array([0.08329189218278059, 0.0014213145240625528, 0.0504764049956106]) 
L515_hand2camera_quat = [0.01521805627198811, 0.00623363612254646, 0.712108725756912, 0.7018765669580811] #xyzw 
L515_intrinsic = np.array([[915.73510742, 0.,           650.41772461],
                           [0.,         916.14764404, 356.6946106],
                           [0.,         0.,           1.]])

camera_calibration = {}
camera_calibration["D415"] = {"pos": D415_hand2camera_pos.tolist(), "quat_xyzw": D415_hand2camera_quat, "intrinsic": D415_intrinsic.tolist()}
camera_calibration["L515"] = {"pos": L515_hand2camera_pos.tolist(), "quat_xyzw": L515_hand2camera_quat, "intrinsic": L515_intrinsic.tolist()}
camera_calibration
#write in a json file
with open('config/camera_calibration.json', 'w') as outfile:
    json.dump(camera_calibration, outfile, indent=1)

In [8]:
#read from camera calin from json file
with open('config/camera_calibration.json') as json_file:
    camera_calibration_read = json.load(json_file)

print(camera_calibration_read["D415"]["pos"])
print(camera_calibration_read["D415"]["quat_xyzw"])
print(camera_calibration_read["D415"]["intrinsic"])

[0.0488546636437146, -0.03384417860749521, 0.0512776975002817]
[0.012961267509189803, -0.0012768531849757236, 0.7052247395136084, 0.708864191484139]
[[925.0, 0.0, 626.0], [0.0, 926.0, 345.0], [0.0, 0.0, 1.0]]


In [9]:
#write pos in json file
with open('config/camera_calibration.json', 'w') as outfile:
    json.dump(views, outfile, indent=1)

In [ ]:
# view_init_pos = [0.17908377, 0.23450877, 0.44783125]
# view_init_orn_wxyz = [0.0030477, 0.92571715, 0.37800879, 0.01215875]
# view_front_pos = [0.36072976, 0.41291873, 0.30648647]
# view_front_orn_wxyz = [ 0.1655683,   0.81494402,  0.28065989, -0.47925298]
# view_right_pos = [0.10118165, 0.47394513, 0.30265239]
# view_right_orn_wxyz = [ 0.3190109,   0.94507174, -0.07051679,  0.00994117]
# view_back_pos = [0.00465632, 0.32838561, 0.26232907]
# view_back_orn_wxyz = [ 0.06700862,  0.96685827, -0.02494526,  0.24509727]

# views_pos = [view_front_pos, view_right_pos, view_back_pos]
# views_orn_wxyz = [view_front_orn_wxyz, view_right_orn_wxyz, view_back_orn_wxyz]

# views = {}
# for i in range(len(views_pos)):
#     views["view"+str(i)] = {"pos": views_pos[i], "orn_wxyz": views_orn_wxyz[i]}
# print(views)